In [1]:
from random_search.mujoco_random_search_learned import MujocoRandomSearchLearned
import click
import json
import gym
import ray 

# @click.command()
# @click.option('--param_file', default='params.json', help='JSON file for exp parameters')

param_file = 'hc.json'  # Replace with the actual path to your JSON file

# Correct usage of the with statement
with open(param_file, 'r') as json_params:
    params = json.load(json_params)
    print(params)
    

exp_identifier = '|'.join('{}={}'.format(key,val) for (key,val) in params.items())
params['exp_id'] = exp_identifier

env = gym.make(params['env_name'])
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

# set policy parameters. Possible filters: 'MeanStdFilter' for v2, 'NoFilter' for v1.
policy_params={'type':'linear',
                'ob_filter': "MeanStdFilter",
                'ob_dim':obs_dim,
                'ac_dim':act_dim}
params["policy_params"] = policy_params
params["dimension"] = obs_dim*act_dim
# ray.init(num_cpus=params["num_workers"], ignore_reinit_error=True, dashboard_host="127.0.0.1")


model = MujocoRandomSearchLearned(params)

/opt/anaconda3/envs/lmrs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-30 18:35:57,567	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[2024-09-30 18:35:57,637] Making new env: HalfCheetah-v1


{'env_name': 'HalfCheetah-v1', 'seed': 4242, 'policy_type': 'linear', 'dir_std': 0.03, 'step_size': 0.02, 'num_workers': 8, 'rollout_length': 1000, 'num_rollouts': 8, 'sampler': 'jacobian', 'n_iter': 2500, 'every_val': 10, 'shift': 0, 'optimizer': 'sgd', 'learning_rate': 0.01, 'num_hidden_dim': 8, 'num_learning_iterations': 10, 'gram_schmidt': True, 'variance_reduced': False, 'filter_corrected': True}


/opt/anaconda3/envs/lmrs/lib/python3.12/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
2024-09-30 18:35:58,886	INFO worker.py:1786 -- Started a local Ray instance.


Sampler is jacobian; so a learner is created


(RayWorkerForMujoco pid=67402) [2024-09-30 18:36:00,342] Making new env: HalfCheetah-v1
(RayWorkerForMujoco pid=67402) /opt/anaconda3/envs/lmrs/lib/python3.12/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(RayWorkerForMujoco pid=67402)   result = entry_point.load(False)


Learner with 102->8->1 using SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: True
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
) with gs:True
TOP-K is 8


In [2]:
import torch
import numpy as np 

num_episodes = 0
model.sampler.current_solution = torch.from_numpy(model.current_solution).view(1,-1).float()

if model.fixed_subspace:
    print('loading')
    model.learner.model.load_state_dict(torch.load('learner_ant'))

In [3]:
iteration = 0
model.ts = iteration + 1
validation_epoch = 10
if iteration % validation_epoch == 0:
    # Evaluate at every validation_epoch
    #print('Evaluation at {}'.format(num_episodes))
    model.evaluate(num_episodes)

sampler = model.sampler
learner = sampler.learner

# jac = sampler.learner.get_jacobian_corrected(sampler.current_solution, normalize=False, gpu=False)

{'AverageRewards': -0.18038001837090475, 'StdRewards': 0.7948992756303357, 'MaxRewards': 1.3396068758980526, 'MinRewards': -1.8139034139063912}


In [4]:
filters = learner.filters
pw = learner.filters[0]
mu = learner.filters[1]
std = learner.filters[2].flatten()
std[std<1e-5] = 1
X = sampler.current_solution
X_mat = X.view(1, pw.shape[0], pw.shape[1])
torch.from_numpy(np.diag(1/std)).float().shape
X_w = torch.matmul(X_mat, torch.from_numpy(np.diag(1/std)).float())

In [9]:
jac = learner.get_jacobian_corrected(X,normalize=False,gpu=False)
jac[1].shape 

torch.Size([3, 102])